# Improved Implementation for Stochastic Linear Regression
with regularization.\
with efficiency improvements.

Todo:
* Stopping Condition

In [1]:
import numpy as np

## Implementation

In [2]:
class LinearRegression:
    """Linear regression model with L2 regularization."""
    
    DEFAULT_EPOCHS = 1000
    DEFAULT_ALPHA = 0.01
    DEFAULT_LAMBDA = 0.0001

    def compute_cost(self, y, y_, Lambda, W, m):
        """Compute cost function with L2 regularization."""
        
        return np.mean((y-y_)**2) + ((np.sum(W**2)) * Lambda/(2*m))
    
    def log_current(self, k, num_out, output_limit, y, y_, Lambda, W, m, b):
        """Log current training information."""
        
        print(f"({k//num_out}/{output_limit}) > Epoch: {k}",
              f"Cost: {self.compute_cost(y,y_,Lambda,W,m):.8f}",
              f"W: {W}",
              f"b: {b:.4f}")
    
    def convergence_test(self, current_cost, past_cost, error_threshold, k):
        # Simple convergence test
        if past_cost - current_cost < error_threshold:
            print(f"\nEpoch: {k} Converged with threshold: {error_threshold}. Returning W and b")
            return True

    def single_step(self, Xi, yi, m, W, b, alpha, Lambda):
        """Perform a single step of gradient descent."""
        
        y_i = np.dot(Xi, W) + b 
        res = yi - y_i
        
        dJ_dW = np.dot(res, Xi)  - Lambda * W
        dJ_db = res.mean()

        W += dJ_dW * alpha / m
        b += dJ_db * alpha

        return W,b
    
    def fit(self, X, y,
            epochs = DEFAULT_EPOCHS,
            alpha = DEFAULT_ALPHA,
            Lambda=DEFAULT_LAMBDA,
            error_threshold = 0.001,
            output_limit=10):
        """Fit the linear regression model to the given data.
        
        Parameter
        ---------
        epochs: int, default=1000
            Number of complete iterations through X

        alpha : float, default=0.01
            Constant Learning Rate

        Lambda : float, default=0.0001
            Rate for l2 Regularization

        output_limit : int, default=10
            Number of iterations to show

        Returns
        -------
        W : numpy.ndarray
            The optimized weights.
        b : numpy.longdouble
            The optimized itercept.
        """
 
        if output_limit<=0:
            raise ValueError("Output limit should be greater than 0")
        
        num_out = epochs//output_limit
        np.set_printoptions(precision=4)
        
        m,n = X.shape
        
        W = np.random.rand(n)
        b = np.random.rand()
        y_ = np.dot(X,W) + b
        past_cost = self.compute_cost(y,y_,Lambda,W,m)
        self.log_current(0, num_out, output_limit, y, y_, Lambda, W, m, b) # Initial Out

        try:
            for k in range(1, epochs+1):
                for i in range(m):
                    W,b = self.single_step(X[i], y[i], m, W, b, alpha, Lambda)
                


                if k % num_out == 0:
                    y_ = np.dot(X,W) + b
                    self.log_current(k, num_out, output_limit, y, y_, Lambda, W, m, b)
                
                
                # Inefficeint and slow to calulate these every epoch.
                y_ = np.dot(X,W) + b
                current_cost = self.compute_cost(y,y_,Lambda,W,m)
                if self.convergence_test(current_cost, past_cost, error_threshold, k):
                    return (W, b)
                past_cost = current_cost

                    
                    
        except KeyboardInterrupt:
            print(f"\nTerminated! Returned: Weights: {W}, Bias: {b}")
            return (W, b)
        return (W,b)

## Usage

In [3]:
m = LinearRegression()

X = np.random.rand(1000,2)
y = 5.55*X[:,0] + 11.22*X[:,1] + 50
m.fit(X, y ,epochs= 1000, alpha = 0.1, error_threshold = 0.0001, output_limit=10)

(0/10) > Epoch: 0 Cost: 3320.64809726 W: [0.5847 0.7841] b: 0.1353
(1/10) > Epoch: 100 Cost: 2.45314503 W: [3.437 6.855] b: 53.8671
(2/10) > Epoch: 200 Cost: 0.43652245 W: [4.6426 9.3857] b: 51.6353
(3/10) > Epoch: 300 Cost: 0.07856247 W: [ 5.1581 10.445 ] b: 50.6954
(4/10) > Epoch: 400 Cost: 0.01451803 W: [ 5.3786 10.8882] b: 50.2996

Epoch: 453 Converged with threshold: 0.0001. Returning W and b


(array([ 5.4383, 11.0062]), 50.19372550284103)